In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!cd /content/gdrive/MyDrive/ComputerVision/Covid19-Detector/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import cv2
import os
import numpy as np

In [ ]:
class DatasetLoader:

  def __init__(self, preprocessors = None):
    
    self.covid_xray = './datasets/images/'
    self.normal_xray_N_train = './datasets/chest_xray/train/NORMAL/'
    self.normal_xray_P_train = './datasets/chest_xray/train/PNEUMONIA/'
    self.patient_information = pd.read_csv('./datasets/patient_id_filename_information.csv').iloc[:,1:]

    self.preprocessors = preprocessors
    if self.preprocessors is None:
      self.preprocessors = []

  def load_data(self):
    
    data, data_label = [],[]

    # iterate through the images and load the images we need for analysis
    for filename in list(set(self.patient_information['filename'])):
      
      image = cv2.imread(os.path.join(self.covid_xray ,filename))
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      if self.preprocessors is not None:
        # convert the image to array and reshape it
        for p in self.preprocessors:
          image = p.preprocess(image)
      # append the processed image to the positive_data list, and the respective label
      data.append(image)
      data_label.append("positive")

    for image in os.listdir(self.normal_xray_N_train):
      image = cv2.imread(os.path.join(self.normal_xray_N_train ,image))
      image = cv2.cvtColor( image, cv2.COLOR_BGR2RGB)
      if self.preprocessors is not None:
        # convert the image to array and reshape it
        for p in self.preprocessors:
          image = p.preprocess(image)

      data.append(image)
      data_label.append("Negative")

    for image in os.listdir(self.normal_xray_P_train):
      image = cv2.imread(os.path.join(self.normal_xray_P_train ,image))
      image = cv2.cvtColor( image, cv2.COLOR_BGR2RGB)
      if self.preprocessors is not None:
        # convert the image to array and reshape it
        for p in self.preprocessors:
          image = p.preprocess(image)
      data.append(image)
      data_label.append("Negative")

    
    # convert the image to array and rescale it to 255
    data = np.array(data) / 255.0
    # convert the labels into an array
    data_label = np.array(data_label)    
    return (data, data_label)